# K723: Data Mining<br>
## Assignment ONE: Clustering 
**Released: Wednesday, October 1, 2025**<br>
**Due on: Monday, October 19, 2025 Before midnight.**

**Please enter your name and student number here:**
<div class="alert alert-block alert-info"> 
    <b>Student Info:</b><BR>
    <ul>
        <li>Student Name: --------</li>
        <li>Student Number: --------</li>
        <li>MAC ID: --------</li>
    </ul>
</div>

**Please follow the following instructions for completing the assignment:**<br>
- **Questions will appear in red alert blocks.**
<div class="alert alert-block alert-danger"> <b>Question number:</b> The body of the question.</div>

    - **For some questions, you only need to modify and run the code.** 
    - **For other questions, you will need to write short report inside green alert blocks.**
<div class="alert alert-block alert-success"> <b>A title: (Optional)</b> Your answer. You can make part of your answer <b>bold</b>, <i>italic</i>, or <u>underlined</u> </div>

- **I will use yellow alert blocks to add helpful comments**
<div class="alert alert-block alert-warning"> <b>Comment title: </b>Comment itself
</div>

- **You will use the following packages for this assignment:**
    - *`numpy`* for mathematical calculations, 
    - *`pandas`* for data handling, 
    - *`scikit-learn`* and *`scipy`* for cluster analysis
       - *`scipy`* to demonstrate hierarchical clustering 
       - *`scikit-learn`* for non-hierarchical clustering
       <br>Both packages provide methods for both types of clustering
    - *`matplotlib`* for visualization.<br>
- **Alongside this notebook you will receive a dataset labeled *`east_west_airlines.csv`*. Please:**
    - Save the dataset and notebook in your *`Working Directory`*
    - Complete the following codes, run them, and use the outcomes to answer questions.
        - Replace ------- with required codes (find them from the notebooks we used in the class and posted on the avenue)


### Dataset Description:

<div class="alert alert-block alert-warning"> <b>The file east_west_airlines.csv contains information on passengers who belong to an airline’s frequent flier program. For each passenger the data include information on their mileage history and on different ways they accrued or spent miles in the last year. The goal is to try to identify clusters of passengers that have similar characteristics for the purpose of targeting different segments for different types of mileage offers.</b>
    <ol>
        <li>Unique ID</li>
        <li>Number of miles eligible for award travel</li>
        <li>Number of miles earned with freq. flyer credit card in the past 12 months</li>
        <li>Number of miles earned with Rewards credit card in the past 12 months</li>
        <li>Number of miles earned with Small Business credit card in the past 12 months</li>
        <li>Number of miles earned from non-flight bonus transactions in the past 12 months</li>
        <li>Number of non-flight bonus transactions in the past 12 months</li>
        <li>Number of days since enrolled in flier program</li>
        <li>Whether that person had award flight (free flight) or not</li>
    </ol>
</div>

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import pairwise
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans
import matplotlib.pylab as plt
import seaborn as sns
from pandas.plotting import parallel_coordinates

%matplotlib inline

-----
-----
### Question 01: Exploring the data and feature scaling (20%)

-----
<div class="alert alert-block alert-danger"> <b>01-01) Please Read data from CSVfile and answer/ the following questions (2%):</b> 
    <ul>
        <li>How many passengers does the dataset have?</li>
        <li>How many features are describing each passenger?</li>
        <li>Legal: 5.905 </li>
        <li>Show <b>the exact</b> first THREE and last THREE observations: </li>
    </ul>
</div>


In [ ]:
name_your_datafram_df = pd.read_csv(-------)
print("Size of the dataset (row, col): ", name_your_datafram_df.-------)
print("\nFirst few obserations\n", name_your_datafram_df.-------)
print("\nLast few obserations\n", name_your_datafram_df.-------)

<div class="alert alert-block alert-success"> <b>Data set size</b> There are ------- records, with ------- features describing each one.</div>


----
<div class="alert alert-block alert-danger"> <b>01-01) Run summary statistics and answer the following questions (3%):</b> 
    <ul>
        <li>What is the range of each four numerical variables?</li>
        <li>Which two variables have the highest correlation?</li>
    </ul>
</div>


In [ ]:
print("\nSummary statistics:\n", name_your_datafram_df.-------)
print("\nCorrelation matrix\n", name_your_datafram_df.-------)

<div class="alert alert-block alert-success"> 
    <b>Range</b> <br>
    <ul>
        <li>Variable 01: ------- </li>
        <li>Variable 01: ------- </li>
        <li>Variable 01: ------- </li>
        <li>Variable 01: ------- </li>
    </ul>
     <br>
    <b>Correlation:</b> The highest correlation is -------, observed between ------- and -------.
</div>

----
<div class="alert alert-block alert-danger"> <b>01-02) Create box plots for all numerical variables (5%): </b><i>(you need to complete and run the first two lines for all variables)</i> 
    <ul>
        <li>Are there any outliers in your dataframe? Which feature(s)</li>
    </ul>
</div>


In [ ]:
plt.figure(-------)
name_your_datafram_df[-------].plot(kind = -------, title = -------)

plt.figure(-------)
name_your_datafram_df[-------].plot(kind = -------, title = -------)


plt.show()

<div class="alert alert-block alert-success"> 
    <b>Outliers:</b> <br>
    There are potential outliers in feature(s): ------- and -------.
</div>

----
<div class="alert alert-block alert-danger"> <b>01-03) Create scatter plots for all pairs of numerical variables (5%): </b><i>(you need to complete and run the first two lines for all pairs of numerical variables)</i> 
    <ul>
        <li>What patterns can you see in scatter plots? <br><i>Compare the plots with the correlation matrix from the question 01-02</i></li>
    </ul>
</div>


In [ ]:
plt.figure(-------)
name_your_datafram_df.plot(kind = -------, x= -------, y= -------, title = -------, alpha = 0.5)

plt.figure(-------)
name_your_datafram_df.plot(kind = -------, x= -------, y= -------, title = -------, alpha = 0.5)


plt.show()

<div class="alert alert-block alert-success"> <b>Patterns based on scatter plots: </b>
    <BR> 
    ---------- ADD YOUR ANSWER HERE -------
</div>

----
<div class="alert alert-block alert-danger"> <b>01-05) Apply feature scaling and explain its importance(5%): </b>
    <ul>
        <li>Apply feature Scaling (2%)</li>
        <li>Why is feature scaling critical before calculating distances between observations and clusters? (2%)</li>
        <li>How can you reverse the feature scaling and retrieve actual values? (1%)</li>
    </ul>
</div>

In [ ]:
name_your_datafram_df2=name_your_datafram_df[[-----, -----, -----, -----, -----]]
name_your_datafram_df2.set_index(------, inplace=True)
name_your_datafram_df2 = name_your_datafram_df2.apply(lambda x: x.astype('float64'))

# pandas uses sample standard deviation
name_your_datafram_df_Norm = (name_your_datafram_df2 - name_your_datafram_df2.-------)/name_your_datafram_df2.-------

<div class="alert alert-block alert-success"> <b>Feature Scaling: </b>
    <BR> 
    ---------- ADD YOUR ANSWER HERE (The importance of feature scaling) -------
    <BR> 
    <BR> 
    ---------- ADD YOUR ANSWER HERE (How to reverse feature scaling to calculate actual values) -------
</div>

-------
-------
### Question 02: Hierarchical Clustering  (25%)
----
<div class="alert alert-block alert-danger"> <b>02-01) Compute normalized distances based on continous variables (5%): </b>
</div>


In [ ]:
# compute normalized distance based on Sales and Fuel Cost
distanceNorm = pairwise.pairwise_distances(name_your_datafram_df_Norm[[-------]], 
                                     metric='euclidean')

<div class="alert alert-block alert-danger"> <b>02-02) Compare hierarchical clustering (10%)</b>
    <ul>
        <li>Use at least three different distance measurement methods (Ward, Centroid, Average). Compare Dendrograms and explain why different methods create different results.</li>
    </ul>
</div>




In [ ]:
Z = linkage(name_your_datafram_df_Norm, method=-------)

fig = plt.figure(figsize=(18, 12), dpi=200)
fig.subplots_adjust(bottom=0.21)
plt.title('Hierarchical Clustering Dendrogram (-------)')
plt.xlabel(--------)
dendrogram(Z, labels=name_your_datafram_df_Norm.index, color_threshold=-------)
plt.axhline(y=-------, color='black', linewidth=1.5, linestyle='dashed')
plt.show()

<div class="alert alert-block alert-warning"> 
    <b>Comparison: </b> Redo the previous block of code in the following Cell(s) for at least one other measurement method and compare the results.<br>
    <b>dpi (dot per inch): </b> a higher value of dpi allows you to have a high-quality image that you can download or open in a new tab. 
</div>

In [ ]:
#Use for second distance method#

In [ ]:
#Use for second distance method#

<div class="alert alert-block alert-success"> <b>Difference between Single Linkage and at least another distance measurement method: </b>
    <BR> 
    ---------- ADD YOUR ANSWER HERE -------
</div>



In [ ]:
memb = fcluster(linkage(name_your_datafram_df_Norm, -------), -------, criterion='maxclust')
memb = pd.Series(memb, index=name_your_datafram_df_Norm.index)
for key, item in memb.groupby(memb):
    print(key, ': ', ', '.join(item.index))

<div class="alert alert-block alert-danger"> <b>02-03) Assign a meaningful label to each cluster based on its characteristics (10%)</b>
    <ul>
        <li>Select a distance measurement method (ward preferably). Use cluster summary (count, mean, min, max) (5%) to give each cluster a meaningful label (5%). </li>
    </ul>
</div>

In [ ]:
name_your_datafram_df['clusterH']=list(memb)
clusterSummaryH = name_your_datafram_df.groupby(['clusterH']).agg({-------
                                                      })
clusterSummaryH

<div class="alert alert-block alert-success"> 
    <b>Cluster Labeling</b> <br>
    <ul>
        <li>Cluster 01 Name: <i>--- Reason why you selected this label ---</i></li>
        <li>Cluster 02 Name: <i>--- Reason why you selected this label ---</i></li>
        <li>Cluster 0X Name: <i>--- Reason why you selected this label ---</i></li>
        <li>Cluster 0Y Name: <i>--- Reason why you selected this label ---</i></li>
    </ul>
</div>

**Add cluster labels to the dataframe using the following block of code:**

In [ ]:
name_your_datafram_df['clusterHLabel']=""
name_your_datafram_df['clusterHLabel'] = np.where(name_your_datafram_df['clusterH'] ==1 , -------,name_your_datafram_df['clusterHLabel'])
name_your_datafram_df['clusterHLabel'] = np.where(name_your_datafram_df['clusterH'] ==2 , -------,name_your_datafram_df['clusterHLabel'])
name_your_datafram_df['clusterHLabel'] = np.where(name_your_datafram_df['clusterH'] ==3 , -------,name_your_datafram_df['clusterHLabel'])

------
------
### Question 03: K-Means Clustering (20%)
----
<div class="alert alert-block alert-danger"> <b>03-01) Perform k-means clustering. Use your judgement (based on previous section) for the number of clusters (5%)</b>
</div>


In [ ]:
# pandas uses sample standard deviation
name_your_datafram_df_KNorm = (name_your_datafram_df2 - name_your_datafram_df2.-------)/name_your_datafram_df2.-------

kmeans = KMeans(n_clusters=-------, random_state=0).fit(name_your_datafram_df_KNorm)

# Cluster membership
memb = pd.Series(kmeans.labels_, index=name_your_datafram_df_KNorm.index)
for key, item in memb.groupby(memb):
    print(key, ': ', ', '.join(item.index))

In [ ]:
withinClusterSS = [0] * -------
clusterCount = [0] * -------
for cluster, distance in zip(kmeans.labels_, kmeans.transform(name_your_datafram_df_KNorm)):
    withinClusterSS[cluster] += distance[cluster]**2
    clusterCount[cluster] += 1
for cluster, withClustSS in enumerate(withinClusterSS):
    print('Cluster {} ({} members): {:5.2f} within cluster'.format(cluster, 
        clusterCount[cluster], withinClusterSS[cluster]))

----
<div class="alert alert-block alert-danger"> <b>03-02) Use the parallel coordinate plot and the cluster summary to characterize and label the clusters. (10%)</b>
</div>

In [ ]:
centroids = pd.DataFrame(kmeans.cluster_centers_, columns=name_your_datafram_df_KNorm.columns)
pd.options.display.max_rows = 7
#in older versions you may use this command: pd.set_option('precision', 3)
print(centroids)

In [ ]:
centroids['cluster'] = ['Cluster {}'.format(i) for i in centroids.index]

plt.figure(figsize=(10,6))
fig.subplots_adjust(right=3)
ax = parallel_coordinates(centroids, class_column='cluster', colormap='Dark2', linewidth=5)
plt.legend(loc='center left', bbox_to_anchor=(0.95, 0.5))
plt.xlim(-0.5,3.5)
centroids

In [ ]:
name_your_datafram_df_KNorm.groupby(kmeans.labels_).mean()

In [ ]:
print(pd.DataFrame(pairwise.pairwise_distances(kmeans.cluster_centers_, metric='euclidean')))

In [ ]:
pd.DataFrame(pairwise.pairwise_distances(kmeans.cluster_centers_, metric='euclidean')).sum(axis=0)

In [ ]:
name_your_datafram_df['clusterK']=list(memb)
clusterSummaryK = name_your_datafram_df.groupby(['clusterK']).agg({-------
                                                      })
clusterSummaryK

<div class="alert alert-block alert-success"> 
    <b>Cluster Labeling</b> <br>
    <ul>
        <li>Cluster 01 Name: <i>--- Reason why you selected this label ---</i></li>
        <li>Cluster 02 Name: <i>--- Reason why you selected this label ---</i></li>
        <li>Cluster 0X Name: <i>--- Reason why you selected this label ---</i></li>
        <li>Cluster 0Y Name: <i>--- Reason why you selected this label ---</i></li>
    </ul>
</div>

**Add cluster labels to the dataframe using the following block of code:**

In [ ]:
name_your_datafram_df['clusterKLabel']=""
name_your_datafram_df['clusterKLabel'] = np.where(name_your_datafram_df['clusterK'] ==0 , -------,name_your_datafram_df['clusterKLabel'])
name_your_datafram_df['clusterKLabel'] = np.where(name_your_datafram_df['clusterK'] ==1 , -------,name_your_datafram_df['clusterKLabel'])
name_your_datafram_df['clusterKLabel'] = np.where(name_your_datafram_df['clusterK'] ==2 , -------,name_your_datafram_df['clusterKLabel'])

<div class="alert alert-block alert-warning"> <b>Centroids vs. Cluster summary:</b><br>
    Centroids are computed based on normalized distance, and you can calculate cluster summaries by reversing the normalization process. You should multiply centroids by the standard deviation of each feature and add the feature's mean to the multiplication.
</div>

In [ ]:
print((centroids*name_your_datafram_df2.std())+name_your_datafram_df2.mean())

----
<div class="alert alert-block alert-danger"> <b>03-03) What is the elbow chart? Develop an elbow chart and cheque if the number of clusters you selected is optimum. (5%).</b>
</div>

In [ ]:
inertia = []
for n_clusters in range(1, 7):
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(name_your_datafram_df_KNorm)
    inertia.append(kmeans.inertia_ / n_clusters)
inertias = pd.DataFrame({'n_clusters': range(1, 7), 'inertia': inertia})
ax = inertias.plot(x='n_clusters', y='inertia')
plt.xlabel('Number of clusters(k)')
plt.ylabel('Average Within-Cluster Squared Distances')
plt.ylim((0, 1.1 * inertias.inertia.max()))
ax.legend().set_visible(False)
plt.show()


<div class="alert alert-block alert-success"> <b>Optimum number of clusters</b> <br>
    ----------- <i> EXPLAIN WHAT ELBOW CHART IS </i> ----------------<br> and
<br>
    ----------- <i> ARGUE IF THE NUMBER OF CLUSTERS THEY USED IS AN OPTIMUM ONE </i> ----------------<br>
</div>

-----
-----
### Question 04: Hierarchical vs. K-Means (15%)
----
<div class="alert alert-block alert-danger"> <b>How do you compare these results with the results from hierarchical clustering? Are they similar or different?(15%)</b>
</div>

In [ ]:
clusterSummaryHK = name_your_datafram_df.groupby(['clusterKLabel','clusterHLabel']).agg({-------
                                                                              })
pd.set_option('display.max_rows', 10)
clusterSummaryHK.head(6)

<div class="alert alert-block alert-success"> <b>Similarities between heirarchical and K-Means clustering</b> <br>
    ----------- <i> REPORT THE OVERLAP BETWEEN K-MEANS AND HEIRARCHICAL CLUSTERING MODELS</i> ----------------<br>
</div>

-----
-----
### Question 05: Your Recommendations (20%)
----
<div class="alert alert-block alert-danger"> <b>As an airline frequent-flyer program manager, decide which clusters to target for incentive offers, explain why you chose this segment (10%).</b>
</div>

----

*WRITE YOUR ANSWER HERE*

----

----
<div class="alert alert-block alert-danger"> <b>Propose types of offers for customers in the selected cluster(s) and explain why (10%).</b>
</div>

----

*WRITE YOUR ANSWER HERE*

----

-----
-----
### Question 06: Your Feedback (5%)
----
<div class="alert alert-block alert-danger"> <b>Feedback:(5%)</b>
    <ul>
        <li>How much time did you spend on this assignment? (1%)</li>
        <li>What prolems did you face? (2%)</li>
        <li>What did you learn? (2%)</li>
    </ul>    
</div>

----

*WRITE YOUR ANSWER HERE*

----